In [ ]:
#install the requisite libraries
!pip install biopython Datasets thefuzz

In [ ]:
import ftplib
from Bio import SeqIO
import glob
import pandas as pd
from datasets import load_dataset

In [ ]:
#initialise list of files to be downloaded from ftp server
file_list = ['Adenoviridae.fna',
 'Adenoviridae.fna',
 'Asfarviridae.fna',
 'Bunyavirales.fna',
 'Caliciviridae.fna',
 'Coronaviridae.fna',
 'Coronaviridae.fna',
 'Filoviridae.fna',
 'Flaviviridae.fna',
 'Hepadnaviridae.fna',
 'Hepeviridae.fna',
 'Herpesviridae.fna',
 'Orthomyxoviridae.fna',
 'Paramyxoviridae.fna',
 'Parvoviridae.fna',
 'Picornaviridae.fna',
 'Pneumoviridae.fna',
 'Polyomaviridae.fna',
 'Poxviridae.fna',
 'Reovirales.fna',
 'Reoviridae.fna',
 'Rhabdoviridae.fna',
 'Togaviridae.fna']

In [ ]:
#helper function
def ftp_list_top_dirs(ftp_session):
    """
       List the top directories on a ftp server

       @param ftp_session: ftp session object

       Returned Value:
          - List of directories and files
           (similar to the Unix command 'ls -l')
    """

    data = []

    # Get the list of files
    ftp_session.dir(data.append)

    return data

In [ ]:
def ftp_get_file(ftp_session, file_name):
    """
         Get a file from a ftp server

         @param ftp_session: ftp session object
         @param file_name: name of the file you want to download
    """
    try:
        ftp_session.retrbinary("RETR " + file_name ,open(file_name, 'wb').write)
    except:
        print("Error - Cannot obtain file: "+ file_name)

In [ ]:
ftp_server = 'ftp.bvbrc.org'
ftp_session= ftplib.FTP(ftp_server)
ftp_session.login()

ftp_session.retrlines('LIST')
ftp_session.cwd('viruses')
for item in file_list:
  ftp_get_file(ftp_session, item)

In [ ]:
for file in glob.glob("*.fna"):
    f_name = str(file).replace('.fna', '')
    viruses = []
    try:
      for record in SeqIO.parse(file,"fasta"):
        record_id = str(record.id)
        record_seq = str(record.seq)
        record_name = str(record.name)
        record_description = str(record.description)
        record_no_of_features = len(record.features)
        record_seq_length = len(str(record.seq))
        virus = [record_id, record_seq, record_name, record_description,
                 record_no_of_features, record_seq_length]
        viruses.append(virus)
    except:
      print(f_name + 'has an error')
    df = pd.DataFrame(viruses, columns = ['id', 'sequence', 'name',
                                          'description', 'features',
                                          'seq_length'])
    df.to_parquet(f'{f_name}.parquet')
    print(f"function call finished for {f_name}")

In [ ]:
# Create a folder for the parquet files and load the dataset
!mkdir dna
!mv ***.parquet /content/dna
dataset = load_dataset("parquet", data_files="/content/dna/*.parquet")

In [ ]:
# If you a huggingface account, you can push the dataset there.
# !huggingface-cli login
# dataset.push_to_hub('Hack90/virus_dna_dataset')